In [7]:
import pandas as pd


In [8]:
ls ./data/nabirds_all_feats.npy
ls ./data/nabirds_all_file_validity.parquet

SyntaxError: invalid syntax (<ipython-input-8-989575874905>, line 1)

In [9]:
#!ln -s /big_fast_drive/orm/nabirds/nabirds/ ./data/nabirds_root

In [14]:
ls './data/nabirds_root/'

bounding_boxes.txt      images/      parts/             sizes.txt
classes.txt             images.txt   photographers.txt  train_test_split.txt
hierarchy.txt           __init__.py  README
image_class_labels.txt  nabirds.py   README~


In [3]:
# !cat './data/nabirds_root/README'

In [451]:
ims

,image_id,image_name
0,0000139e-21dc-4d0c-bfe1-4cae3c85c829,0817/0000139e21dc4d0cbfe14cae3c85c829.jpg
1,0000d9fc-4e02-4c06-a0af-a55cfb16b12b,0860/0000d9fc4e024c06a0afa55cfb16b12b.jpg
2,00019306-9d83-4334-b255-a447742edce3,0900/000193069d834334b255a447742edce3.jpg
3,0001afd4-99a1-4a67-b940-d419413e23b3,0645/0001afd499a14a67b940d419413e23b3.jpg
4,000332b8-997c-4540-9647-2f0a8495aecf,0929/000332b8997c454096472f0a8495aecf.jpg
...,...,...
48557,fff86e8b-795f-400a-91e8-565bbb8c453a,0891/fff86e8b795f400a91e8565bbb8c453a.jpg
48558,fff926d7-ccad-4788-839e-97af2dd99372,0660/fff926d7ccad4788839e97af2dd99372.jpg
48559,fffa33ef-a765-408d-8d66-6efc7f504c71,0492/fffa33efa765408d8d666efc7f504c71.jpg
48560,ffff0d87-bc84-4ef2-a47e-a4bfa48502ce,0372/ffff0d87bc844ef2a47ea4bfa48502ce.jpg


In [15]:
ims = pd.read_csv('./data/nabirds_root/images.txt', delimiter=' ', names=['image_id', 'image_name'])
splits = pd.read_csv('./data/nabirds_root/train_test_split.txt', delimiter=' ', names=['image_id', 'is_training_image'])
#classes = pd.read_csv('./data/nabirds_root/classes.txt', delimiter=' ', names=['class_id', 'class_name'])
lns = open('./data/nabirds_root/classes.txt', 'r').readlines()
classes = pd.DataFrame([l.replace('\n','').split(' ', 1) for l in lns], columns=['class_id', 'class_description'])
classes = classes.assign(class_id=classes.class_id.astype('int'))
image_labels = pd.read_csv('./data/nabirds_root/image_class_labels.txt', delimiter=' ', names=['image_id', 'class_id'])

In [16]:
class_hierarchy = pd.read_csv('./data/nabirds_root/hierarchy.txt', 
                              delimiter=' ', 
                              names=['child_class_id', 'parent_class_id'])

In [17]:
# import networkx as nx
# ch = class_hierarchy[['child_class_id', 'parent_class_id']]
# edges = ch.apply(tuple,axis=1).values
# dg = nx.DiGraph()
# dg.add_nodes_from(classes.class_id.values)
# dg.add_edges_from(edges)

In [18]:
degs = pd.DataFrame(dg.in_degree, columns=['node_id', 'in_degree'])
leaf_nodes = degs[degs.in_degree == 0].node_id.values

NameError: name 'dg' is not defined

In [204]:
assert set(image_labels.class_id.values) - set(leaf_nodes) == set()
## all labels are leaf nodes

In [221]:
paths = []
for sn in dg.nodes:
    pth = [sn]
    while True:
        last = pth[-1]
        if last == 0:
            break
        e = dg.out_edges(last)
        assert len(e) == 1
        next_id = list(e)[0][1]
        pth.append(next_id)
    paths.append(pth)
    
word_paths = []
for p in paths:
    pn = classes.class_description.values[p]
    word_paths.append((p[0], pn.tolist()))

In [255]:
id2paths = dict(word_paths)

In [259]:
id2simple_name = {i:(id2paths[i][1], for i in leaf_nodes}

In [290]:
idtable = []
for k,v in id2paths.items():
    if k in leaf_nodes:
        assert v[0].startswith(v[1])
        base = v[1]
        modifier = v[0][len(v[1]):].strip()
        if modifier != '':
            assert modifier.startswith('(')
            assert modifier.endswith(')')
            modifier = modifier[1:-1]
            
        idtable.append((k,base,modifier,v[2:]))

In [294]:
simplified_class = pd.DataFrame(idtable, columns=['class_id', 'class_name', 'class_modifier', 'supercategory'])

In [29]:
simplified_class.groupby(['class_name']).size()

NameError: name 'simplified_class' is not defined

In [296]:
simplified_class[simplified_class.class_name == 'Yellow-rumped Warbler']

,class_id,class_name,class_modifier,supercategory
291,747,Yellow-rumped Warbler,Breeding Myrtle,"[Wood-Warblers, Perching Birds, Birds]"
342,798,Yellow-rumped Warbler,Winter/juvenile Myrtle,"[Wood-Warblers, Perching Birds, Birds]"
502,958,Yellow-rumped Warbler,Breeding Audubon's,"[Wood-Warblers, Perching Birds, Birds]"
553,1009,Yellow-rumped Warbler,Winter/juvenile Audubon's,"[Wood-Warblers, Perching Birds, Birds]"


In [303]:
gt = image_labels.merge(ims, left_on='image_id', right_on='image_id')
gt = gt.merge(simplified_class[['class_id', 'class_name', 'class_modifier']], left_on='class_id', right_on='class_id')
#bc = gt.set_index(['image_name', 'class_name'])#['class_id'].unstack(level=-1).fillna(-1) >=0).astype('float')

In [308]:
gttmp = gt[['image_name', 'class_name']].reset_index().set_index(['image_name', 'class_name'])['index']

#.unstack(level=-1)

In [314]:
simple_gt = gttmp.unstack(level=-1, fill_value=-1).ge(0).astype('float')

In [316]:
(simple_gt.sum(axis=1) == 1).all()

True

In [319]:
simple_gt.sum().sort_values()

class_name
Cackling Goose            31.0
Bank Swallow              35.0
Northwestern Crow         36.0
Pacific Wren              37.0
Scaled Quail              39.0
                         ...  
American Goldfinch       240.0
Red-tailed Hawk          298.0
European Starling        308.0
Dark-eyed Junco          362.0
Yellow-rumped Warbler    435.0
Length: 404, dtype: float64

In [30]:
df = pd.read_parquet('./data/nabirds_most_specific_classes_gt.parquet')

In [355]:
idpg = fetch_page(id_url)

In [358]:
base = '/html/body/div[2]/main/section[4]/ul/'
p = idpg.xpath(base  + 'li[1]/div')[0]

In [379]:
# p.cssselect('aside > img')[0].attrib['data-interchange']
# p.cssselect('aside > p').text_content()

In [380]:
item = fkeys[0]

In [390]:
item.cssselect('div > aside > div.video-responsive')

[]

In [410]:
x = 'ah ...   '

In [470]:
del entry

In [504]:
from tqdm.auto import tqdm
from lxml import html
import requests
import urllib
import pandas as pd

def fetch_page(url):
    header_dict = { ## seems to return 403 without this string.
        'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.61 Safari/537.36',
    }

    page = requests.get(url, headers=header_dict)
    pg = html.fromstring(page.text)
    return pg

def get_id_info(species_name):
    sp = species_name
    cname = sp.replace("'","").replace(" ", "_")
    comp_url = f'https://www.allaboutbirds.org/guide/{cname}/species-compare'
    id_url = f'https://www.allaboutbirds.org/guide/{cname}/id'
    
    idpg = fetch_page(id_url)
    if len(idpg.cssselect('ul.four-keys')) > 0:
        ## some pages don't have this
        id_vals = process_basic_id(idpg)
    else:
        id_vals = None
    
    compg = fetch_page(comp_url)
    
    if len(compg.cssselect('div.main-species')) > 0:
        main_id = get_main(compg)
    else:
        main_id = []
        
    if len(compg.cssselect('div.similar-species')) > 0:
        similar_ids = get_similar(compg)
    else:
        similar_ids = []
    
    return {'species':sp, 
            'cname': cname,
            'features_url':id_url,
            'features':id_vals, 
            'similar_to_url':comp_url,
            'examples':main_id, 
            'similar_to':similar_ids}


def process_id_listitem(item):
    ps = item.cssselect('div > article > p')
    text = []
    for p in ps:
        t = p.text_content()
        if t != '':
            tt = t.rstrip('. ')
            text.append(tt)
    
    desc = '. '.join(text)
    imgs = item.cssselect('div > aside > img')

    ## desc missing image 'https://www.allaboutbirds.org/guide/Black_Tern/id'
    if len(imgs) > 0:
        img = imgs[0]
        image_alt = img.attrib['alt']
        image_url = img.attrib['data-interchange'].split(',')[0].replace('[','')
    else:
        image_alt = None
        image_url = None
        
    return {'description':desc, 'image_url':image_url, 'image_alt':image_alt}


def process_basic_id(idpg):
    fkeys = idpg.cssselect('ul.four-keys')[0].getchildren()
    ks = ['size_and_shape', 'color_pattern']
    lis = zip(['size_and_shape', 'color_pattern'], fkeys)
    desc = {}
    for (info,item) in lis:
        desc[info] = process_id_listitem(item)
        
    return desc

def process_entry(entry):
    link = entry.cssselect('h3 > a')
    if len(link) > 0:
        species = link[0].attrib['href'].split('/')[2]
    else:
        return {}
        
    diff_desc = entry.cssselect('div.annotation-txt > p')[0].text_content()
    if diff_desc == '':
        diff_desc = None

    attributes = entry.cssselect('div.annotation-txt > h5 ')
    if len(attributes) == 0:
        attribute = None
    else:
        attribute = attributes[0].text_content()
        
    img = entry.cssselect('img')[0]
    image_alt = img.attrib['alt']
    image_url = img.attrib['data-interchange'].split(',')[0].replace('[','')
    return {'sim_species':species, 
            'description':diff_desc, 
            'attribute':attribute, 
            'image_url':image_url,
            'image_alt':image_alt
           }


def process_img_item(item):
    img = item.cssselect('img')[0]
    img_alt = img.attrib['alt']
    img_url = img.attrib['data-interchange'].split(',')[0].replace('[', '')
    
    desc = item.cssselect('div.annotation-txt > p')[0].text_content() 
    if desc== '':
        desc = None
    
    attributes = item.cssselect('div.annotation-txt > h5 ')
    if len(attributes) == 0:
        attribute = None
    else:
        attribute = attributes[0].text_content()
    
    return {'image_url':img_url, 
            'image_alt':img_alt,
            'description':desc,
            'attribute':attribute}

def get_similar(pg):
    sim_species = pg.cssselect(' div.similar-species > div ')[0].getchildren()
    similar = []
    for entry in sim_species:
        ent = process_entry(entry)
        similar.append(ent)
    return similar


def get_main(pg):
    items = pg.cssselect('div.main-species > div.compare-main > li')
    descs = []
    for item in items:
        imgs = item.cssselect('img') ## 0 or 1
        videos = item.cssselect('div.video-responsive') ## 0 or 1

        if len(imgs) > 0:
            desc = process_img_item(item)
            descs.append(desc)
        elif len(videos) > 0:
            continue ## not doing anything about videos right now
            
    return descs

In [475]:
import pickle

In [476]:
import json

In [511]:
#json.dump(data, open('./data/nabirds_meta.json', 'w'))

In [512]:
for ent in data:
    break

In [519]:
str('hi')

'hi'

In [541]:
def get_image_urls(ent):
    urls = []
    for ln in json.dumps(ent).split(' '):
        if ln.find('.jpg"')>= 0:
            url = ln.strip(',"')
            urls.append(url)
            
    return urls

In [544]:
eiu = get_image_urls(ent)

In [545]:
all_urls = set()

In [548]:
all_urls

{1, 2}

In [549]:
all_urls = set()
for ent in tqdm(data):
    ent_urls = get_image_urls(ent)
    all_urls.update(ent_urls)

  0%|          | 0/404 [00:00<?, ?it/s]

In [550]:
len(all_urls)

5144

In [505]:
species = df.columns.values
#data = []
for c in tqdm(species[39:]):
    info = get_id_info(c)
    data.append(info)

  0%|          | 0/365 [00:00<?, ?it/s]

In [552]:
import os

In [588]:
local_root = '/big_fast_drive2/orm/bird_guide/images/'
remote_root = 'https://www.allaboutbirds.org/'

# '/big_fast_drive2/orm/bird_guide/images/'
os.makedirs(local_root, exist_ok=True)

In [ ]:
opener = urllib.request.build_opener()
opener.addheaders = [('User-agent', 'Mozilla/5.0')]
urllib.request.install_opener(opener)

errs = []
for url in tqdm(all_urls):
    try:
        path = url.replace(remote_root, '')
        parts = path.split('/')
        prefix = '/'.join(parts[:-1])
        fname = parts[-1]
        os.makedirs(f'{local_root}/{prefix}', exist_ok=True)
        opath = f'{local_root}/{prefix}/{fname}'
        [fname, _] = urllib.request.urlretrieve(url, filename=opath)
        print(f'downloaded {url} to {opath}')
    except KeyboardInterrupt as e:
        raise e
    except Exception as e:
        print(f'failed to download image {url}... exception: {e}')
        errs.append(url)
        continue

In [593]:
### what's the evaluation task:
#### eg you have a video of birds
### eg. you have a bird dataset and want to retrieve images using domain vocabulary

### out of sample images of birds. eg. can you do tasks better on them?

In [ ]:
#### 1. system to help leverage existing domain knowledge beyond what is available in model
#### base exp: can I fine tune clip to work well with birds
### which task will i do? one shot classification using scientific name seems not too useful.
####
#### similarity search from images is an easy task to measure.
### so is object detection.
#### developing a video pipeline may be a good task as well.

### attribute search:
### birds with beak such and such

## what is the interactive component here...: refine by attribute. 'like this image but with more x'
### generating prompts


#### 2. system to help searches with interactive attribute refinement.
### alternative 2: interactive attribute refinement

In [594]:
import pytorch_lightning as pl
import transformers
import clip

In [625]:
def load_model():
    variant ="ViT-B/32"
    jit = False
    model,_ = clip.load(variant, device='cpu',  jit=jit)
    return model

class StringEncoder(object):
    def __init__(self, model, device):
        self.device = device #next(iter(clean_weights.items()))[1].device
        self.model = model.to(device)
        self.original_weights = copy.deepcopy(model.state_dict())
        self.reset() # dont update the original model
        
    def encode_string(self, string):
        model = self.model.eval()
        with torch.no_grad():
            ttext = clip.tokenize([string])
            text_features = model.encode_text(ttext.to(self.device))
            text_features = text_features / text_features.norm(dim=-1, keepdim=True)
            return text_features.detach().cpu().numpy()
        
    def reset(self):
        self.model.load_state_dict(copy.deepcopy(self.original_weights))

In [626]:
import torch
class MappableDataset(torch.utils.data.Dataset):
    def __init__(self, ds, tx):
        self.ds = ds
        self.tx = tx
        
    def map(self, tx):
        return MappableDataset(self, tx=tx)
        
    def __getitem__(self, idx):
        return self.tx(self.ds[idx])
        
    def __len__(self):
        return len(self.ds)
    
import torchvision.transforms as T

In [627]:
from torchvision.transforms import Compose, Resize, CenterCrop, ToTensor, Normalize

def img_transform(n_px):
    return Compose([
        Resize(n_px, interpolation=T.InterpolationMode.BICUBIC),
        CenterCrop(n_px),
        lambda image: image.convert("RGB"),
        ToTensor(),
        Normalize((0.48145466, 0.4578275, 0.40821073), (0.26862954, 0.26130258, 0.27577711)),
    ])

image_tx = img_transform(224)


def tab_tx(tup):
    title_tok = clip.tokenize(tup['description'].split('.')[0])[0]
    #desc_tok = clip.tokenize(tup['description'])[0]
    #genre_toks = clip.tokenize(tup['genre'])[0]
    #     genres = list(map(lambda s : s.strip(), tup['genre'].split(',')))    
    
    return {'title':title_tok, 
            #'description':desc_tok,
            #'genre':genre_toks, 
           'image':image_tx(tup['image'])}


from torch import nn
import torch.nn.functional as F
import math

class ContrastLoss(nn.Module):
    def __init__(self, log_temp):
        super().__init__()
        self.log_temp = log_temp
    
    def loss(self, logits):
        target = torch.arange(logits.shape[0]).long().to(logits.device)
        return F.cross_entropy(logits, target, reduction='mean')
    
    def forward(self, vecs1, vecs2):
        assert vecs1.shape == vecs2.shape
        # cosine similarity as logits
        logits = (math.exp(self.log_temp) * vecs1) @ vecs2.t()
        return self.loss(logits) + self.loss(logits.t())

In [673]:
class MultiColumnEncoder(pl.LightningModule):
    def __init__(self, encoder_dict, log_temp=1.):
        super().__init__()
        self.encoders = nn.ModuleDict(encoder_dict.items())
        self.loss_fn = ContrastLoss(log_temp)
        
        
    def forward(self, batch):
        batch_out = {}
        for k,v in batch.items():
            assert k in self.encoders
            vout = self.encoders[k](v)
            vout = vout / (vout.norm(dim=-1,keepdim=True) + 1e-6)
            batch_out[k] = vout
            
        return batch_out
            
    def basic_step(self, batch, train_or_val):
        batch_out = self.forward(batch)
        loss = self.loss_fn(batch_out['image'], batch_out['description'])
        self.log(f'{train_or_val}_loss', loss)
        return loss
    
    def validation_step(self, batch, batch_idx):
        return self.basic_step(batch, 'val')
    
    def training_step(self, batch, batch_idx):
        return self.basic_step(batch, 'train')
    
    def configure_optimizers(self):
        return torch.optim.AdamW(self.parameters(), lr=0.0001)

In [667]:
from clip.model import CLIP
import copy

class CLIPTextEmbedding(nn.Module):
    def __init__(self, clip_mod):
        super().__init__()
        
                
        members = ['token_embedding', 
                   'transformer', 
                   'ln_final']
        
        for m in members:
            self.add_module(m, copy.deepcopy(getattr(clip_mod, m)))
            
        
        self.positional_embedding = nn.Parameter(copy.deepcopy(clip_mod.positional_embedding))
        self.text_projection = nn.Parameter(copy.deepcopy(clip_mod.text_projection))
        
        self.dtype = clip_mod.dtype
        self.vocab_size = clip_mod.vocab_size
        self.context_length = clip_mod.context_length

    def forward(self, text):
        x = self.token_embedding(text).type(self.dtype)  # [batch_size, n_ctx, d_model]

        x = x + self.positional_embedding.type(self.dtype)
        x = x.permute(1, 0, 2)  # NLD -> LND
        x = self.transformer(x)
        x = x.permute(1, 0, 2)  # LND -> NLD
        x = self.ln_final(x).type(self.dtype)

        # x.shape = [batch_size, n_ctx, transformer.width]
        # take features from the eot embedding (eot_token is the highest number in each sequence)
        x = x[torch.arange(x.shape[0]), text.argmax(dim=-1)] @ self.text_projection

        return x      

In [668]:
class DataModule(pl.LightningDataModule):
    def __init__(self, ds, batch_size, val_size, num_workers=4):
        super().__init__()

        self.ds = ds
        idxs = np.random.permutation(len(ds))
        
        train_idxs = idxs[:-val_size]
        val_idxs = idxs[-val_size:]
        
        self.train_ds = torch.utils.data.Subset(ds, indices=train_idxs)
        self.val_ds = torch.utils.data.Subset(ds, indices=val_idxs)
        self.batch_size = batch_size
        self.num_workers = num_workers
    
    def train_dataloader(self):
        return torch.utils.data.DataLoader(self.train_ds, batch_size=self.batch_size, 
                                           shuffle=True, num_workers=self.num_workers)
    
    def val_dataloader(self):
        return torch.utils.data.DataLoader(self.val_ds, batch_size=self.batch_size, 
                                           shuffle=True, num_workers=self.num_workers)

In [609]:
as_records = []
for ent in tqdm(data):
    species = ent['species']
    cname = ent['cname']
    
    if ent['features'] is not None:
        for k,v in ent['features'].items():
            v = v.copy()
            v['cname'] = cname
            v['species'] = species 
            v['entry_type'] = k
            as_records.append(v)
    
    for ex in ent['examples']:
        ex = ex.copy()
        ex['cname'] = cname
        ex['species'] = species 
        ex['entry_type'] = 'example'
        as_records.append(ex)
        
    for ex in ent['similar_to']:
        ex = ex.copy()
        ex['cname'] = cname
        ex['species'] = species
        ex['entry_type'] = 'similar_to'
        as_records.append(ex)


  0%|          | 0/404 [00:00<?, ?it/s]

In [631]:
local_root = '/big_fast_drive2/orm/bird_guide/images/'
remote_root = 'https://www.allaboutbirds.org/'

In [614]:
bird_df = pd.DataFrame.from_records(as_records)

In [697]:
bird_df.to_parquet('./data/bird_guide_df.parquet')

In [618]:
bird_df.image_url.shape[0]

6162

In [698]:
# bad_paths = []
# path_ok = np.ones(bird_df.shape[0], dtype=np.bool8)
# for (i,url) in enumerate(bird_df.image_url.values):
#     if url is None or url != url:
#         path_ok[i] = False
#         continue

#     p = local_root + url.replace('https://www.allaboutbirds.org','')
#     if not os.path.exists(p):
#         path_ok[i] = False
        
        
# bird_df = bird_df.assign(path_ok=path_ok)

In [699]:
bdf1 = bird_df[(bird_df.entry_type == 'example') & (~bird_df.description.isna()) & (bird_df.path_ok)]

In [658]:
import PIL 
class BirdGuideDataset(MappableDataset):
    def __init__(self, image_root, df):
        self.image_root = image_root
        self.df = df
        
    def __len__(self):
        return self.df.shape[0]
    
    def __getitem__(self, idx):
        ent = self.df.iloc[idx]
        relpath = ent.image_url.replace('https://www.allaboutbirds.org','')
        path = self.image_root + relpath
        return {'image':PIL.Image.open(path), 'description':ent.description, 'species':ent.species}

In [693]:
bgds = BirdGuideDataset(image_root='/big_fast_drive2/orm/bird_guide/images/', df=bdf1)

In [694]:
txds = bgds.map(lambda tup : {'description':clip.tokenize(tup['description'].split('.')[0])[0], 
                              'image':image_tx(tup['image']) })

In [696]:
txds[1234]

{'description': tensor([49406, 12944, 42274,   631,  6547,   998,  4348,   593, 15145, 41046,
         49407,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0]),
 'image': tensor([[[-0.9748, -0.9748, -0.9748,  ..., -1.0039, -1.0039, -1.0039],
          [-0.9748, -0.9748, -0.9748,  ..., -1.0039, -1.0185, -1.0185],
          [-0.9748, -0.9748, -0.9748,  ..., -1.0185, -1.0331, -1.0331],
          ...,
          [-0.8872, -0.8288, -0.7996,  ..., -0.5076, -0.5222, -0.4930],
          [-0.9164, -0.8580, -0.7996,  ..., -0.5076, -0.5222

In [662]:
mod = load_model()

/nvme_drive/vsms4_no_tensorflow/lib/python3.8/site-packages/torchvision/transforms/transforms.py:280: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  warnings.warn(


In [671]:
import numpy as np

In [701]:
bdf1.shape

(2766, 9)

In [703]:
mce = MultiColumnEncoder({'image':mod.visual, 'description':CLIPTextEmbedding(mod)})
#imdb_samps = imdb_samps.assign(got_image=pd.Series(valids))
dm = DataModule(txds, batch_size=96, val_size=1024, num_workers=8)
chk = pl.callbacks.ModelCheckpoint(monitor='val_loss', mode='min')
tb_logger = pl.loggers.TensorBoardLogger("logs_bird_ft/", name='try0')
trainer = pl.Trainer(gpus=[1],max_epochs=30, 
#                      max_steps=200*41, 
#                      val_check_interval=200,
                     num_sanity_val_steps=0,
                     logger=[tb_logger], callbacks=[chk])

trainer.validate(mce, datamodule=dm)
trainer.fit(mce, datamodule=dm)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


Validating: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 VALIDATE RESULTS
{'val_loss': 8.618917465209961}
--------------------------------------------------------------------------------


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name     | Type         | Params
------------------------------------------
0 | encoders | ModuleDict   | 151 M 
1 | loss_fn  | ContrastLoss | 0     
------------------------------------------
151 M     Trainable params
0         Non-trainable params
151 M     Total params
605.109   Total estimated model params size (MB)
/nvme_drive/vsms4_no_tensorflow/lib/python3.8/site-packages/pytorch_lightning/trainer/data_loading.py:326: UserWarning: The number of training samples (19) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  rank_zero_warn(


Training: -1it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

In [705]:
## now do something with this new model...
## eg. a task on a different dataset

In [706]:
import torchvision

In [827]:
cat /big_fast_drive2/orm/CUB_200_2011/CUB_200_2011/

The Caltech-UCSD Birds-200-2011 Dataset

For more information about the dataset, visit the project website:

  http://www.vision.caltech.edu/visipedia

If you use the dataset in a publication, please cite the dataset in
the style described on the dataset website (see url above).

Directory Information
---------------------

- images/
    The images organized in subdirectories based on species. See 
    IMAGES AND CLASS LABELS section below for more info.
- parts/
    15 part locations per image. See PART LOCATIONS section below 
    for more info.
- attributes/
    322 binary attribute labels from MTurk workers. See ATTRIBUTE LABELS 
    section below for more info.



IMAGES AND CLASS LABELS:
Images are contained in the directory images/, with 200 subdirectories (one for each bird species)

------- List of image files (images.txt) ------
The list of image file names is contained in the file images.txt, with each line corresponding to one image:

<image_id> <image_name>
---------------

In [831]:
class_attr = pd.read_csv('/big_fast_drive2/orm/CUB_200_2011/CUB_200_2011/attributes/class_attribute_labels_continuous.txt',
                              #names=['_id', 'attribute_id', 'is_present', 'certainty_id', 'time'], 
                         header=None,
                        delim_whitespace=True)


In [833]:
pd.merge(attribute_labels

,image_id,attribute_id,is_present,certainty_id,time
0,1,1,0,3,27.708
1,1,2,0,3,27.708
2,1,3,0,3,27.708
3,1,4,0,3,27.708
4,1,5,1,3,27.708
...,...,...,...,...,...
3677851,11788,308,1,4,4.989
3677852,11788,309,0,4,8.309
3677853,11788,310,0,4,8.309
3677854,11788,311,0,4,8.309


In [832]:
class_attr

,0,1,2,3,4,5,6,7,8,9,...,302,303,304,305,306,307,308,309,310,311
0,0.000000,2.919708,1.459854,0.000000,59.854015,26.277372,3.649635,5.839416,0.000000,1.639344,...,0.000000,0.000000,21.276596,11.347518,2.12766,31.914894,53.571429,5.357143,21.428571,19.642857
1,4.411765,4.411765,2.941176,1.470588,57.352941,27.205882,0.000000,2.205882,0.000000,0.000000,...,0.000000,2.255639,2.255639,94.736842,0.00000,0.000000,77.142857,3.809524,10.476190,8.571429
2,0.000000,3.973510,3.311258,0.000000,70.860927,14.569536,5.298013,1.986755,0.000000,0.000000,...,2.205882,0.000000,38.970588,2.941176,0.00000,16.176471,71.028037,0.934579,7.476636,20.560748
3,0.000000,1.459854,10.218978,0.000000,0.000000,0.000000,8.759124,25.547445,54.014599,5.839416,...,0.000000,0.000000,88.571429,0.000000,0.00000,2.142857,84.126984,2.380952,3.174603,10.317460
4,0.000000,0.000000,3.225806,0.000000,1.075269,0.000000,0.000000,32.258065,63.440860,0.000000,...,0.000000,0.000000,82.727273,5.454545,0.00000,4.545455,88.461538,0.000000,3.846154,7.692308
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,2.702703,12.837838,0.000000,0.000000,0.000000,0.000000,76.351351,8.108108,0.000000,0.000000,...,0.000000,0.000000,0.000000,1.973684,0.00000,29.605263,14.379085,22.875817,56.209150,6.535948
196,18.300654,11.764706,0.000000,3.267974,0.000000,0.000000,60.784314,5.882353,0.000000,0.000000,...,0.000000,0.000000,35.443038,4.430380,0.00000,22.784810,16.783217,25.174825,32.167832,25.874126
197,10.759494,36.708861,0.000000,7.594937,0.632911,0.000000,43.670886,0.632911,0.000000,0.000000,...,0.000000,0.000000,14.375000,13.125000,0.00000,41.875000,18.791946,44.295302,18.791946,18.120805
198,0.000000,2.068966,0.000000,2.068966,0.000000,0.000000,88.275862,7.586207,0.000000,0.000000,...,0.000000,1.324503,3.973510,0.000000,0.00000,19.205298,12.413793,34.482759,33.103448,20.000000


In [839]:
#/big_fast_drive2/orm/CUB_200_2011/CUB_200_2011/image_class_labels.


In [774]:
images = pd.read_csv('/big_fast_drive2/orm/CUB_200_2011/CUB_200_2011/images.txt', 
                             names=['image_id', 'image_name'], delim_whitespace=True)

#cat /big_fast_drive2/orm/CUB_200_2011/CUB_200_2011/attributes/certainties.txt
certainties = ['not visible','guessing', 'probably', 'definitely']

attribute_labels = pd.read_csv('/big_fast_drive2/orm/CUB_200_2011/CUB_200_2011/attributes/image_attribute_labels.txt',
                              names=['image_id', 'attribute_id', 'is_present', 'certainty_id', 'time'], delim_whitespace=True)
attributes = pd.read_csv('/big_fast_drive2/orm/CUB_200_2011/attributes.txt', names=['attribute_id', 'attribute_name'], delim_whitespace=True)

In [792]:
att_labels = pd.merge(attribute_labels, attributes, left_on='attribute_id', right_on='attribute_id')

In [795]:
attributes[attributes.attribute_col == 'has_wing_pattern']

,attribute_id,attribute_name,attribute_col,attribute_variant
308,309,has_wing_pattern::solid,has_wing_pattern,solid
309,310,has_wing_pattern::spotted,has_wing_pattern,spotted
310,311,has_wing_pattern::striped,has_wing_pattern,striped
311,312,has_wing_pattern::multi-colored,has_wing_pattern,multi-colored


In [ ]:
'/big_fast_drive2/orm/CUB_200_2011/CUB_200_2011/images/' + images[images.image_id == 8049].image_name.iloc[0]

In [848]:
# PIL.Image.open()

In [853]:
#### task: richer attribute based search
#### task: help users building a pipeline quickly

In [ ]:
### finetuned vs. pre-computed.

In [ ]:
sys.__

In [840]:
### get some initial result 
#### prepare revision code for use by student. also, for use by myself.

In [842]:
images

,image_id,image_name
0,1,001.Black_footed_Albatross/Black_Footed_Albatr...
1,2,001.Black_footed_Albatross/Black_Footed_Albatr...
2,3,001.Black_footed_Albatross/Black_Footed_Albatr...
3,4,001.Black_footed_Albatross/Black_Footed_Albatr...
4,5,001.Black_footed_Albatross/Black_Footed_Albatr...
...,...,...
11783,11784,200.Common_Yellowthroat/Common_Yellowthroat_00...
11784,11785,200.Common_Yellowthroat/Common_Yellowthroat_00...
11785,11786,200.Common_Yellowthroat/Common_Yellowthroat_00...
11786,11787,200.Common_Yellowthroat/Common_Yellowthroat_00...


In [845]:
### what do you want to do now?

In [847]:
## write an evaluation task:
### search based on attribute?

In [797]:
att_labels_pivot = att_labels.pivot(index='image_id', columns='attribute_name', values='is_present')

In [798]:
att_labels_pivot

attribute_name,has_back_color::black,has_back_color::blue,has_back_color::brown,has_back_color::buff,has_back_color::green,has_back_color::grey,has_back_color::iridescent,has_back_color::olive,has_back_color::orange,has_back_color::pink,...,has_wing_color::yellow,has_wing_pattern::multi-colored,has_wing_pattern::solid,has_wing_pattern::spotted,has_wing_pattern::striped,has_wing_shape::broad-wings,has_wing_shape::long-wings,has_wing_shape::pointed-wings,has_wing_shape::rounded-wings,has_wing_shape::tapered-wings
image_id,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,1,0,1,0,0,0
4,0,0,1,0,0,0,0,0,0,0,...,0,0,1,0,0,1,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11784,0,0,0,0,0,1,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
11785,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
11786,0,0,0,0,0,0,0,1,0,0,...,1,0,0,0,1,0,0,0,1,0


In [776]:
#cat '/big_fast_drive2/orm/CUB_200_2011/CUB_200_2011/attributes/certainties.txt'

In [789]:
att0 = attributes.attribute_name.apply(lambda s : s.split('::')[0])
att1 = attributes.attribute_name.apply(lambda s : s.split('::')[1])

In [791]:
attributes = attributes.assign(attribute_col=att0, attribute_variant=att1)

In [770]:
#attribute_labels

In [746]:
cat /big_fast_drive2/orm/CUB_200_2011/CUB_200_2011/README

The Caltech-UCSD Birds-200-2011 Dataset

For more information about the dataset, visit the project website:

  http://www.vision.caltech.edu/visipedia

If you use the dataset in a publication, please cite the dataset in
the style described on the dataset website (see url above).

Directory Information
---------------------

- images/
    The images organized in subdirectories based on species. See 
    IMAGES AND CLASS LABELS section below for more info.
- parts/
    15 part locations per image. See PART LOCATIONS section below 
    for more info.
- attributes/
    322 binary attribute labels from MTurk workers. See ATTRIBUTE LABELS 
    section below for more info.



IMAGES AND CLASS LABELS:
Images are contained in the directory images/, with 200 subdirectories (one for each bird species)

------- List of image files (images.txt) ------
The list of image file names is contained in the file images.txt, with each line corresponding to one image:

<image_id> <image_name>
---------------

In [707]:
torchvision.datasets.DatasetFolder('/big_fast_drive2/orm/CUB_200_2011/', extensio)

TypeError: __init__() missing 1 required positional argument: 'loader'

In [719]:
#'/big_fast_drive2/orm/CUB_200_2011/CUB_200_2011/attributes/image_attribute_labels.txt'
#pd.read_csv('/big_fast_drive2/orm/CUB_200_2011/CUB_200_2011/attributes/image_attribute_labels.txt')

In [723]:
bird_df[[ 'image_url', 'description', 'species', 'image_alt', 'entry_type']]

,image_url,description,species,image_alt,entry_type
0,https://www.allaboutbirds.org/guide/assets/pho...,"A large sparrow with a short, conical bill, an...",Abert's Towhee,Abert's Towhee,size_and_shape
1,https://www.allaboutbirds.org/guide/assets/pho...,"Abert’s Towhee is grayish-brown above, pinkish...",Abert's Towhee,Abert's Towhee,color_pattern
2,https://www.allaboutbirds.org/guide/noindex/ph...,None,Abert's Towhee,Abert's Towhee Adult,example
3,https://www.allaboutbirds.org/guide/noindex/ph...,"Large, ground-dwelling sparrow with a thick bi...",Abert's Towhee,Abert's Towhee Adult,example
4,https://www.allaboutbirds.org/guide/noindex/ph...,"Large, long-tailed sparrow with a restricted r...",Abert's Towhee,Abert's Towhee Adult,example
...,...,...,...,...,...
6157,https://www.allaboutbirds.org/guide/assets/pho...,Female and immature Blackburnian Warblers have...,Yellow-throated Warbler,Blackburnian Warbler Female/immature is simila...,similar_to
6158,https://www.allaboutbirds.org/guide/assets/pho...,Males have a bright orange (not yellow) throat...,Yellow-throated Warbler,Blackburnian Warbler Adult male is similar to ...,similar_to
6159,https://www.allaboutbirds.org/guide/assets/pho...,Pine Warblers have a yellow-green (not gray) b...,Yellow-throated Warbler,Pine Warbler Adult male is similar to Yellow-t...,similar_to
6160,https://www.allaboutbirds.org/guide/assets/pho...,Female/immature Magnolia Warblers lack the Yel...,Yellow-throated Warbler,Magnolia Warbler Female/immature is similar to...,similar_to


In [735]:
## task 0: learn a species classifier. (note: are the species already in the encyclopedia?)
### task 1: attribute classification (CUB has attribute data)
### taks 2: part detection (both cub and nabirds have part boxes)

In [728]:
tmp = bird_df[[ 'image_url', 'description', 'species', 'image_alt', 'entry_type']]
tmp = tmp[tmp.entry_type == 'similar_to']

In [733]:
tmp.description.iloc[0]

"Canyon Towhee has a buffy tinged face, not the dark face of Abert's Towhee. Canyon lives in desert scrub whereas Abert's occurs in brushy thickets along streams."

In [734]:
tmp.description.iloc[-1]

'Adult male Magnolia Warblers have a black necklace that Yellow-throated Warblers lack. '

In [717]:
#ls '/big_fast_drive2/orm/CUB_200_2011/CUB_200_2011/attributes/'

In [720]:
#### observations. a model pre-trained with a large web corpus is available.
### multiple large domain databases/encyclopedias are also available. these are an alternative
## to creating your own captioned corpus.

In [737]:
cat /big_fast_drive2/orm/CUB_200_2011/attributes.txt

1 has_bill_shape::curved_(up_or_down)
2 has_bill_shape::dagger
3 has_bill_shape::hooked
4 has_bill_shape::needle
5 has_bill_shape::hooked_seabird
6 has_bill_shape::spatulate
7 has_bill_shape::all-purpose
8 has_bill_shape::cone
9 has_bill_shape::specialized
10 has_wing_color::blue
11 has_wing_color::brown
12 has_wing_color::iridescent
13 has_wing_color::purple
14 has_wing_color::rufous
15 has_wing_color::grey
16 has_wing_color::yellow
17 has_wing_color::olive
18 has_wing_color::green
19 has_wing_color::pink
20 has_wing_color::orange
21 has_wing_color::black
22 has_wing_color::white
23 has_wing_color::red
24 has_wing_color::buff
25 has_upperparts_color::blue
26 has_upperparts_color::brown
27 has_upperparts_color::iridescent
28 has_upperparts_color::purple
29 has_upperparts_color::rufous
30 has_upperparts_color::grey
31 has_upperparts_color::yellow
32 has_upperparts_color::olive
33 has_upperparts_color::green
34 has_upperparts_color::pink
35 has_upperparts_color::orange
36 has_upperparts_

In [ ]:
# image | caption model
### difference model: leverage descriptions of differences to learn a better model.